In [217]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import math
import cv2
import numpy as np
import os

from dotenv import load_dotenv
from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt

In [218]:
load_dotenv()

True

In [219]:
# Define seed to have the same images rotation through the dataset creation

In [220]:
random.seed(42)
images_folder = os.environ.get('IMAGES_FOLDER')
saving_folder_images = os.environ.get('SAVING_FOLDER_IMAGES')
annotation_folder = os.environ.get('ANNOTATION_FOLDER')
annotation_file = os.environ.get('ANNOTATION_FILE')

In [221]:
def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

In [222]:
df = pd.read_csv(annotation_file)
df = df[df.degrees.notna()]

In [223]:
for imageId in df.imageId:
    filename = imageId + '.jpg'
    original_image = Image.open(dirname+'/'+filename)
    image_height, image_width = original_image.height, original_image.width
    original_rotation = df.loc[df.imageId == imageId, 'degrees'].item()
    angle = float(random.randrange(0, 36000)/100)
    angle = (angle + original_rotation) % 360
    df.loc[df.imageId == imageId, 'degrees'] = angle
    degreesToRadians = angle*(math.pi/180)

    # Clockwise rotation
    im = original_image.rotate(-angle)
    image_rotated = np.array(im)
    numpy_image_rotated_cropped = crop_around_center(
        image_rotated,
        *largest_rotated_rect(
            image_width,
            image_height,
            degreesToRadians
        )
    )
    image_rotated_cropped = Image.fromarray(numpy_image_rotated_cropped)
    image_rotated_cropped.save(saving_folder_images+'/'+filename)
print(str(df.count()) + ' images rotated')

imageId    95
degrees    95
dtype: int64 images rotated


In [224]:
now = datetime.now()
now_date = str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'-'+str(now.hour)+'_'+str(now.minute)+'_'+str(now.second)
df.to_csv(annotation_folder+"/rotated-images-"+now_date+".csv", index=False)

In [225]:
df.head(108)

,imageId,degrees
0,2007_000027,114.82
1,2007_000032,16.99
2,2007_000033,180.24
3,2007_000039,161.94
4,2007_000042,147.19
...,...,...
102,2007_001594,350.30
103,2007_001595,173.03
104,2007_001602,277.84
106,2007_001627,264.93


In [226]:
df.describe()

,degrees
count,95.000000
mean,169.070842
std,95.250396
min,14.850000
25%,91.630000
50%,164.410000
75%,240.100000
max,356.050000
